# Feature Engineering & Transformation

This section covers:
1. Normalization / scaling.
2. Feature selection & dimensionality reduction.
3. Derived feature creation (e.g., per-trait means, variance).
4. Optional numerosity reduction (sampling or PCA).

In [ ]:
import pandas as pd

# Visualization
import seaborn as sns

# Preprocessing & stats
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression

# Settings
pd.set_option("display.max_columns", None)
sns.set(style="whitegrid")

In [ ]:
import pandas as pd
import numpy as np

# Load feature-engineered dataset
df = pd.read_csv("../data/processed/feature_engineered.csv")

print("✅ Dataset loaded successfully")

✅ Dataset loaded successfully


In [ ]:
# Dataset shape
print("Shape:", df.shape)

# Column names
print("\nColumns:")
for col in df.columns:
    print(col)

Shape: (1011478, 130)

Columns:
EXT1
EXT2
EXT3
EXT4
EXT5
EXT6
EXT7
EXT8
EXT9
EXT10
EST1
EST2
EST3
EST4
EST5
EST6
EST7
EST8
EST9
EST10
AGR1
AGR2
AGR3
AGR4
AGR5
AGR6
AGR7
AGR8
AGR9
AGR10
CSN1
CSN2
CSN3
CSN4
CSN5
CSN6
CSN7
CSN8
CSN9
CSN10
OPN1
OPN2
OPN3
OPN4
OPN5
OPN6
OPN7
OPN8
OPN9
OPN10
EXT1_E
EXT2_E
EXT3_E
EXT4_E
EXT5_E
EXT6_E
EXT7_E
EXT8_E
EXT9_E
EXT10_E
EST1_E
EST2_E
EST3_E
EST4_E
EST5_E
EST6_E
EST7_E
EST8_E
EST9_E
EST10_E
AGR1_E
AGR2_E
AGR3_E
AGR4_E
AGR5_E
AGR6_E
AGR7_E
AGR8_E
AGR9_E
AGR10_E
CSN1_E
CSN2_E
CSN3_E
CSN4_E
CSN5_E
CSN6_E
CSN7_E
CSN8_E
CSN9_E
CSN10_E
OPN1_E
OPN2_E
OPN3_E
OPN4_E
OPN5_E
OPN6_E
OPN7_E
OPN8_E
OPN9_E
OPN10_E
dateload
screenw
screenh
introelapse
testelapse
endelapse
IPC
country
lat_appx_lots_of_err
long_appx_lots_of_err
EXT
NEU
AGR
CON
OPN
EXT_bin
NEU_bin
AGR_bin
CON_bin
OPN_bin
EXT_mean
EXT_std
NEU_mean
NEU_std
AGR_mean
AGR_std
CON_mean
CON_std
OPN_mean
OPN_std


In [ ]:
# Preview first 5 rows
df.head()

# Optional: random sample of 10 rows to see scaling and engineered features
df.sample(10, random_state=42)

,EXT1,EXT2,EXT3,EXT4,EXT5,EXT6,EXT7,EXT8,EXT9,EXT10,EST1,EST2,EST3,EST4,EST5,EST6,EST7,EST8,EST9,EST10,AGR1,AGR2,AGR3,AGR4,AGR5,AGR6,AGR7,AGR8,AGR9,AGR10,CSN1,CSN2,CSN3,CSN4,CSN5,CSN6,CSN7,CSN8,CSN9,CSN10,OPN1,OPN2,OPN3,OPN4,OPN5,OPN6,OPN7,OPN8,OPN9,OPN10,EXT1_E,EXT2_E,EXT3_E,EXT4_E,EXT5_E,EXT6_E,EXT7_E,EXT8_E,EXT9_E,EXT10_E,EST1_E,EST2_E,EST3_E,EST4_E,EST5_E,EST6_E,EST7_E,EST8_E,EST9_E,EST10_E,AGR1_E,AGR2_E,AGR3_E,AGR4_E,AGR5_E,AGR6_E,AGR7_E,AGR8_E,AGR9_E,AGR10_E,CSN1_E,CSN2_E,CSN3_E,CSN4_E,CSN5_E,CSN6_E,CSN7_E,CSN8_E,CSN9_E,CSN10_E,OPN1_E,OPN2_E,OPN3_E,OPN4_E,OPN5_E,OPN6_E,OPN7_E,OPN8_E,OPN9_E,OPN10_E,dateload,screenw,screenh,introelapse,testelapse,endelapse,IPC,country,lat_appx_lots_of_err,long_appx_lots_of_err,EXT,NEU,AGR,CON,OPN,EXT_bin,NEU_bin,AGR_bin,CON_bin,OPN_bin,EXT_mean,EXT_std,NEU_mean,NEU_std,AGR_mean,AGR_std,CON_mean,CON_std,OPN_mean,OPN_std
829475,1.862315,-1.348293,1.412432,-1.744888,1.351179,-1.150482,0.875684,-1.916391,1.514271,-0.434830,-0.218665,0.678415,-1.610570,1.866938,-0.669161,-0.640861,-0.821089,-1.261494,-0.848150,-1.349401,-0.194368,1.028874,1.361614,0.954918,-0.235922,0.205662,-1.074563,0.279594,0.175806,0.374237,-1.116678,0.767934,0.986827,-0.499617,-1.284107,0.823310,0.263468,1.345750,-0.163306,1.346037,0.294703,-0.072271,-0.007657,-0.003682,0.203701,0.999629,-0.991220,-0.952469,-2.078787,-0.947774,2604.0,1086.0,1392.0,757.0,2362.0,1219.0,6692.0,1784.0,1428.0,4672.0,2061.0,4126.0,1128.0,2907.0,2278.0,2197.0,821.0,2122.0,2675.0,1190.0,4786.0,1322.0,930.0,1436.0,7679.0,2373.0,2417.0,1138.0,1163.0,1909.0,1488.0,1565.0,2886.0,3826.0,2041.0,1809.0,1654.0,4584.0,9151.0,1950.0,1709.0,5104.0,3729.0,1956.0,1360.0,2050.0,4123.0,953.0,3604.0,1596.0,2018-07-21 02:32:52,1440.0,900.0,26.0,129.0,5,1,US,37.4135,-122.1312,-0.497428,1076.45,-0.663579,1549.400,-0.571043,Low,Low,Low,Low,Low,0.042100,1.504761,-0.487404,1.047542,0.287585,0.710004,0.246962,0.971594,-0.355583,0.880521
331567,0.274994,-1.348293,1.412432,-0.119509,-0.223345,-0.332238,-1.273134,1.248791,-1.469270,-0.434830,0.528330,1.497342,0.995003,1.065941,0.119077,0.115767,-1.598144,-1.261494,-0.848150,-1.349401,-0.942068,1.028874,-0.992226,0.954918,-1.092182,1.030858,-1.074563,1.202275,1.041452,1.309218,1.450830,0.045974,0.986827,-1.301251,0.290295,0.823310,0.263468,-1.293624,1.412424,0.385968,-2.322180,0.828834,-0.007657,-0.923157,-0.818406,-0.815405,0.015552,-1.753537,0.847060,-0.947774,9429.0,5864.0,2755.0,5417.0,2916.0,5416.0,5950.0,3249.0,7316.0,8150.0,8523.0,3081.0,3917.0,7254.0,5018.0,7084.0,5549.0,3915.0,2913.0,1884.0,3017.0,5586.0,1781.0,2115.0,4130.0,3702.0,3767.0,2817.0,3351.0,2565.0,7430.0,10552.0,6614.0,5147.0,3269.0,5615.0,3816.0,5718.0,11468.0,9954.0,3731.0,6259.0,3886.0,6331.0,4250.0,6866.0,6217.0,2466.0,3267.0,2769.0,2017-02-03 12:24:55,1280.0,1024.0,44.0,255.0,16,1,US,44.4984,-84.592,-0.116314,2458.35,-0.474776,3480.900,-0.041980,High,High,Low,High,High,-0.226440,1.004349,-0.073573,1.120841,0.246656,1.099920,0.306422,0.970914,-0.589667,1.026336
236917,0.274994,1.683593,0.583972,0.693181,-0.223345,0.486006,0.159411,1.248791,-0.723385,0.336442,0.528330,0.678415,0.126478,1.866938,-0.669161,-1.397489,-1.598144,-1.261494,-0.073486,-1.349401,-0.942068,0.142692,-0.992226,0.057798,1.476597,1.030858,0.715895,0.279594,1.041452,0.374237,0.594994,-1.397947,0.986827,-1.301251,1.077495,-1.301625,0.263468,-1.293624,-0.163306,1.346037,0.294703,1.729938,-0.007657,1.835269,-1.840513,-0.815405,0.015552,-0.952469,-1.103505,0.032768,4596.0,5386.0,1827.0,1658.0,3530.0,2301.0,3830.0,3186.0,5812.0,2386.0,4420.0,2666.0,2960.0,3796.0,3316.0,2213.0,2387.0,1730.0,3533.0,2219.0,5059.0,1846.0,2383.0,1743.0,5797.0,2453.0,6816.0,4376.0,4220.0,2431.0,4066.0,3044.0,1887.0,3207.0,3676.0,4019.0,2580.0,1739.0,1743.0,1728.0,4994.0,4230.0,2572.0,4304.0,3080.0,2044.0,4526.0,5467.0,6440.0,3630.0,2016-10-12 16:10:50,1920.0,1080.0,7.0,171.0,21,1,AU,-37.9367,145.0342,-0.373908,1463.30,-0.369172,1385.900,-0.168523,Low,Low,Medium,Low,Medium,0.451966,0.682833,-0.314901,1.136352,

In [ ]:
trait_cols = ["EXT", "NEU", "AGR", "CON", "OPN"]  # Adjust if column names differ
print("Descriptive statistics for Big Five traits:")
df[trait_cols].describe().T

Descriptive statistics for Big Five traits:


,count,mean,std,min,25%,50%,75%,max
EXT,1011478.0,7.373227e-17,1.000000,-0.583334,-0.370653,-0.241090,-0.024411,7.497549
NEU,1011478.0,2.397707e+03,1874.574232,613.100000,1414.000000,1875.000000,2635.050000,13555.210500
AGR,1011478.0,7.912731e-17,1.000000,-0.958642,-0.510313,-0.269028,0.118021,6.173437
CON,1011478.0,2.775640e+03,2176.262912,653.327000,1610.300000,2158.100000,3080.650000,15568.256500
OPN,1011478.0,-1.384728e-16,1.000000,-0.941267,-0.511595,-0.274149,0.110331,6.069753


In [16]:
bin_cols = [f"{col}_bin" for col in trait_cols if f"{col}_bin" in df.columns]

for col in bin_cols:
    print(f"\nValue counts for {col}:")
    print(df[col].value_counts())



Value counts for EXT_bin:
EXT_bin
Low       337176
High      337157
Medium    337145
Name: count, dtype: int64

Value counts for NEU_bin:
NEU_bin
Low       337163
High      337160
Medium    337155
Name: count, dtype: int64

Value counts for AGR_bin:
AGR_bin
Low       337168
High      337155
Medium    337155
Name: count, dtype: int64

Value counts for CON_bin:
CON_bin
Low       337176
High      337153
Medium    337149
Name: count, dtype: int64

Value counts for OPN_bin:
OPN_bin
Low       337183
Medium    337150
High      337145
Name: count, dtype: int64


In [20]:
from sklearn.decomposition import PCA

# Select only numeric columns from your item features + engineered stats
numeric_item_cols = df[item_cols].select_dtypes(include=np.number).columns

# Apply PCA to retain 95% variance
pca = PCA(n_components=0.95, random_state=42)
pca_components = pca.fit_transform(df[numeric_item_cols])

# Results
print("✅ PCA applied on numeric item features")
print(f"Original numeric item features: {len(numeric_item_cols)}")
print(f"PCA components to retain 95% variance: {pca_components.shape[1]}")
print("Cumulative variance explained (95% target):", np.sum(pca.explained_variance_ratio_))


✅ PCA applied on numeric item features
Original numeric item features: 59
PCA components to retain 95% variance: 46
Cumulative variance explained (95% target): 0.95033122297687
